In [1]:
# Common imports
import os
import imageio
import numpy as np
import cv2
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (16, 8)

In [2]:
# Common parameters
grid_size = 30
patch_size = 64
global_y_valid = 534
# Resolution before was 1412x913
# Resolution current is 
# Forensic Graph: 1024x682
# ManTraNet: 1024x682
# CAT-Net: 256x172
# [Todo] compute y_valid

In [33]:
# Helper functions
def get_pred_aggregate(grid_size, patch_size, im, im_label, return_labels=False, manip=False):
    '''
        Convert pixel-wise prediction map to a nxn map
    '''
    im_h, im_w, _ = im.shape
    pred_aggregate = np.empty((grid_size, grid_size))
    for idx_i, i in enumerate(np.linspace(0, im_h - patch_size, grid_size).astype(int)):
        for idx_j, j in enumerate(np.linspace(0, im_w - patch_size, grid_size).astype(int)):
            pred_aggregate[idx_i, idx_j] = np.mean(im[i:i+patch_size, j:j+patch_size, :].astype(float))/255
    return pred_aggregate

    
def get_mAP_splice(y_valid, out, fname):
    
    # load manip mask
    manip_mask_dir = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v2c-div4-crop/'
    manip_mask_fname = fname[9:45]+'.png'
    manip_mask_path = os.path.join(manip_mask_dir, manip_mask_fname)
    h, w = out.shape
    # Make it binary
    manip_label = (cv2.imread(manip_mask_path)[:global_y_valid, :, 0] == 255).astype('uint8')
    manip_label = cv2.resize(manip_label, (w, h), cv2.INTER_NEAREST)
    manip_label = manip_label.astype(out.dtype)
#     manip_label = manip_label[100:, 200:800]
#     out = out[100:, 200:800]
    gt = manip_label.ravel()
    estimate = out.ravel()
    
#     print('GT stats: ', gt.min(), gt.max())
#     print('Estimate stats: ', estimate.min(), estimate.max())
    score = metrics.average_precision_score(gt, estimate)    
    return score

def get_mAP_color(y_valid, out, fname):
    
    # load manip mask
    manip_dir = '/data/vision/phillipi/gan-training/totem/resources/totems/data/manip_patch/'
    manip_mask_path = os.path.join(manip_dir, fname[:11]+'mask.png')
    h, w = out.shape
    # Make it binary
    manip_label = (cv2.imread(manip_mask_path)[:global_y_valid, :, 0] == 255).astype('uint8')
#     print(manip_label.shape)
#     print(out.shape)
    manip_label = cv2.resize(manip_label, (w, h), cv2.INTER_NEAREST)
    manip_label = manip_label.astype(out.dtype)
    
    gt = manip_label.ravel()
    estimate = out.ravel()
#     print('GT stats: ', gt.min(), gt.max())
#     print('Estimate stats: ', estimate.min(), estimate.max())
    score = metrics.average_precision_score(gt, estimate)    
    return score, manip_label, out

def get_mAP_CAF(y_valid, out, fname):
    
    # load manip mask
    manip_mask_dir = '/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/'
    manip_mask_fname = fname[:-4]+'_mask.png'
    manip_mask_path = os.path.join(manip_mask_dir, manip_mask_fname)
    h, w = out.shape
    # Make it binary
    manip_label = (cv2.imread(manip_mask_path)[:global_y_valid, :, 0] == 255).astype('uint8')
    manip_label = cv2.resize(manip_label, (w, h), cv2.INTER_NEAREST)
    manip_label = manip_label.astype(out.dtype)
#     manip_label = manip_label[100:, 200:800]
#     out = out[100:, 200:800]
    gt = manip_label.ravel()
    estimate = out.ravel()
    
#     print('GT stats: ', gt.min(), gt.max())
#     print('Estimate stats: ', estimate.min(), estimate.max())
    score = metrics.average_precision_score(gt, estimate)    
    return score
  

### Forensic Graph, splice (no undistort+crop)

In [51]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/forensic-graph/localization/splice'
fnames = sorted(os.listdir(folder))

In [52]:
curr_width = 1024
curr_patch_size = round(patch_size/1440.*curr_width)
curr_y_valid = round(global_y_valid/1440.*curr_width)
curr_height = round(960/1440.*curr_width)

In [53]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    # plt.imshow(pred_aggregate_upsample, cmap='jet')
    scores.append(get_mAP_splice(curr_y_valid, pred_aggregate_upsample, fname))

In [54]:
print("mAP is: ", np.mean(scores), len(scores))

mAP is:  0.04588093630405767 10


In [55]:
scores = []
for fname in fnames:
     # This line loads Jacob's results
    pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_height), cv2.INTER_CUBIC)[:curr_y_valid, ...]
    scores.append(get_mAP_splice(curr_y_valid, pred_aggregate_upsample, fname))

In [56]:
# Table 2, Row 1, Column 2
print("Jacob's mAP is: ", round(np.mean(scores), 3), len(scores))

Jacob's mAP is:  0.037 10


### ManTraNet, splice (no undistort+crop)

In [57]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/ManTraNet/localization/splice'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]

In [58]:
curr_width = 1024
curr_patch_size = round(patch_size/1440.*curr_width)
curr_y_valid = round(global_y_valid/1440.*curr_width)
curr_height = round(960/1440.*curr_width)

In [59]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    # plt.imshow(pred_aggregate_upsample, cmap='jet')
    scores.append(get_mAP_splice(curr_y_valid, pred_aggregate_upsample, fname))

In [60]:
# Table 2, Row 2, Column 2
print("mAP is: ",round(np.mean(scores), 3), len(scores))

mAP is:  0.151 10


### CAT-Net, splice (no undistort+crop)

In [270]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/CAT-Net/localization/splice'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]
print(fnames)

['JT8A8283_JT8A8292_content_aware_fill_002_mask.png', 'JT8A8283_JT8A8299_content_aware_fill_001_mask.png', 'JT8A8287_JT8A8291_content_aware_fill_001_mask.png', 'JT8A8287_JT8A8297_content_aware_fill_001_mask.png', 'JT8A8290_JT8A8291_content_aware_fill_001_mask.png', 'JT8A8290_JT8A8293_content_aware_fill_001_mask.png', 'JT8A8293_JT8A8291_content_aware_fill_001_mask.png', 'JT8A8293_JT8A8299_content_aware_fill_001_mask.png', 'JT8A8297_JT8A8291_content_aware_fill_001_mask.png', 'JT8A8299_JT8A8292_content_aware_fill_002_mask.png']


In [271]:
curr_width = 256
curr_patch_size = round(patch_size/1440.*curr_width)
curr_y_valid = round(global_y_valid/1440.*curr_width)
curr_height = round(960/1440.*curr_width)
print(curr_patch_size, curr_y_valid)


11 95


In [274]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    # plt.imshow(pred_aggregate_upsample, cmap='jet')
#     print(pred_aggregate_upsample.sum())
    scores.append(get_mAP_splice(curr_y_valid, pred_aggregate_upsample, fname))

In [275]:
print("mAP is: ", np.mean(scores), len(scores))

mAP is:  0.23668748565648906 10


### Forensic Graph, color patch

In [43]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/forensic-graph/localization/patch'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]

In [44]:
curr_width = 1024
curr_patch_size = round(patch_size/1412.*curr_width)
curr_y_valid = round(global_y_valid/1412.*curr_width)
curr_height = round(913/1412.*curr_width)

In [160]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    scores.append(get_mAP_color(curr_y_valid, pred_aggregate_upsample, fname)[0])

In [161]:
print("mAP is: ", np.mean(scores), len(scores))

mAP is:  0.05870117040536329 21


In [45]:
scores = []
for fname in fnames:
    # This line loads Jacob's results
    pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/notebooks/detection_results_patch/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_height), cv2.INTER_CUBIC)[:curr_y_valid, ...]
    scores.append(get_mAP_color(curr_y_valid, pred_aggregate_upsample, fname)[0])

In [46]:
# Table 2, Row 1, Column 3
print("Jacob's mAP is: ", round(np.mean(scores), 3), len(scores))

Jacob's mAP is:  0.8 21


In [173]:
scores = []
for fname in fnames:
    # This line loads our joint+L1 results
    path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_patch/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
    if not os.path.exists(path):
        continue
    pred_aggregate_upsample = np.load(path)
    scores.append(get_mAP_color(global_y_valid, pred_aggregate_upsample, fname)[0])

In [175]:
# Because not limited to protected area
print("Best mAP is: ", np.mean(scores), len(scores))

Best mAP is:  0.6086876276733647 15


### ManTraNet, color patch

In [47]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/ManTraNet/localization/patch'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]

In [48]:
curr_width = 1024
curr_patch_size = round(patch_size/1412.*curr_width)
curr_y_valid = round(global_y_valid/1412.*curr_width)
curr_height = round(913/1412.*curr_width)

In [49]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    scores.append(get_mAP_color(curr_y_valid, pred_aggregate_upsample, fname)[0])

In [50]:
# Table 2, Row 2, Column 3
print("mAP is: ", round(np.mean(scores), 3), len(scores))

mAP is:  0.295 21


### CAT-Net, color patch

In [220]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/CAT-Net/localization/patch'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]

In [221]:
curr_width = 256
curr_patch_size = round(patch_size/1412.*curr_width)
curr_y_valid = round(global_y_valid/1412.*curr_width)
curr_height = round(913/1412.*curr_width)

In [222]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    scores.append(get_mAP_color(curr_y_valid, pred_aggregate_upsample, fname)[0])

In [223]:
print("mAP is: ", np.mean(scores), len(scores))

mAP is:  0.2475677955163484 21


### ManTraNet, CAF

In [39]:
folder = '/data/vision/phillipi/cortex/code/forensics-baseline/results/ManTraNet/localization_full/CAF'
fnames = sorted(os.listdir(folder))
fnames = [fname for fname in fnames if not fname.startswith('.')]

In [40]:
curr_width = 1024
curr_patch_size = round(patch_size/1412.*curr_width)
curr_y_valid = round(global_y_valid/1412.*curr_width)
curr_height = round(913/1412.*curr_width)

In [41]:
scores = []
for fname in fnames:
    pred = imageio.imread(os.path.join(folder, fname))[:curr_y_valid, ...]
    pred_aggregate = get_pred_aggregate(grid_size, curr_patch_size, pred, None, return_labels=False, manip=False)
#     # This line loads Jacob's results
#     pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
#     # This line loads our joint+L1 results
#     path = f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems/notebooks/detection_results_splice/v1_L1_30x30_results/{fname[:-4]+'_result.npy'}"
#     if not os.path.exists(path):
#         continue
#     pred_aggregate = np.load(path)
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_y_valid), cv2.INTER_CUBIC)
    scores.append(get_mAP_CAF(curr_y_valid, pred_aggregate_upsample, fname))

/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8283_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8287_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8291_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8292_content_aware_fill_002_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8293_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8297_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirt

In [42]:
# Table 2, Row 2, Column 1
print("mAP is: ", round(np.mean(scores), 3), len(scores))

mAP is:  0.181 7


In [36]:
scores = []
for fname in fnames:
    # This line loads Jacob's results
    pred_aggregate = np.load(f"/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/notebooks/detection_results_CAF/jacob_30x30_results/{fname[:-4]+'_result.npy'}")
    pred_aggregate_upsample = cv2.resize(pred_aggregate, (curr_width, curr_height), cv2.INTER_CUBIC)[:curr_y_valid, ...]
    scores.append(get_mAP_CAF(curr_y_valid, pred_aggregate_upsample, fname))

/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8283_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8287_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8291_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8292_content_aware_fill_002_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8293_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirtualhome/totem/totems-misc/data/content-aware-v3-div4-crop/JT8A8297_content_aware_fill_001_mask.png
/data/vision/torralba/virtualhome/realvirtualhome/realvirt

In [38]:
# Table 2, Row 1, Column 1
print("Jacob's mAP is: ", round(np.mean(scores), 3), len(scores))

Jacob's mAP is:  0.036 7


In [ ]:
c